In [196]:
import sqlite3
import pandas as pd 
connect_sqlite = sqlite3.connect("valorant.sqlite")
games = pd.read_sql_query("SELECT * FROM Games", con=connect_sqlite)
game_scoreboard = pd.read_sql_query("SELECT * FROM Game_Scoreboard", con=connect_sqlite)


In [197]:
game_scoreboard.dropna(inplace=True)
games.dropna(inplace=True)

In [198]:
merged = pd.merge(game_scoreboard[["GameID", "Agent","TeamAbbreviation"]], games[["GameID","Team1","Team2","Winner"]], on="GameID")

### Finding uncorrect matches

In [199]:
import collections
indexs = collections.defaultdict(int)

for id,row in merged.iterrows():
    indexs[row["GameID"]] += 1

errors = []
for key,value in indexs.items():
    if value != 10:
        errors.append(key)

errors

['60644', '60567', '60441', '60621', '60607', '53322', '53321']

In [200]:
merged.drop(merged.query("GameID == '60644' | GameID == '60567' | GameID == '60441' | GameID == '60621' | GameID == '60607' | GameID == '53322' | GameID == '53321'").index, inplace=True)
merged.reset_index(inplace=True)

### Syncing team abbreviation w/ team name

In [201]:
import numpy as np
merged["thisteam"] = np.nan
for id,row in merged.iterrows():
    if id%10 <= 4:
        merged.loc[id,"thisteam"] = row["Team1"]
    else:
        merged.loc[id,"thisteam"] = row["Team2"]

C:\Users\user\AppData\Local\Temp\ipykernel_16980\1245820542.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Booster Seat Gaming' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged.loc[id,"thisteam"] = row["Team1"]


In [210]:
merged

,index,GameID,Agent,TeamAbbreviation,Team1,Team2,Winner,thisteam
0,0,60894,jett,Boos,Booster Seat Gaming,Pho Real,Booster Seat Gaming,Booster Seat Gaming
1,1,60894,chamber,Boos,Booster Seat Gaming,Pho Real,Booster Seat Gaming,Booster Seat Gaming
2,2,60894,sova,Boos,Booster Seat Gaming,Pho Real,Booster Seat Gaming,Booster Seat Gaming
3,3,60894,viper,Boos,Booster Seat Gaming,Pho Real,Booster Seat Gaming,Booster Seat Gaming
4,4,60894,skye,Boos,Booster Seat Gaming,Pho Real,Booster Seat Gaming,Booster Seat Gaming
...,...,...,...,...,...,...,...,...
3315,3362,53281,sage,GEN,100 Thieves,Gen.G,100 Thieves,Gen.G
3316,3363,53281,jett,GEN,100 Thieves,Gen.G,100 Thieves,Gen.G
3317,3364,53281,sova,GEN,100 Thieves,Gen.G,100 Thieves,Gen.G
3318,3365,53281,omen,GEN,100 Thieves,Gen.G,100 Thieves,Gen.G


In [248]:
pickrate = merged["Agent"].value_counts().reset_index()
pickrate["percent"] = pickrate["count"] / 664 * 100
pickrate

,Agent,count,percent
0,jett,550,82.831325
1,sova,536,80.722892
2,astra,425,64.006024
3,viper,356,53.614458
4,skye,321,48.343373
5,killjoy,251,37.801205
6,cypher,162,24.397590
7,sage,136,20.481928
8,raze,127,19.126506
9,reyna,116,17.469880


In [215]:
wins = merged[merged["Winner"] == merged["thisteam"]]["Agent"].value_counts()
loses = merged[merged["Winner"] != merged["thisteam"]]["Agent"].value_counts()

In [237]:
winrate = pd.merge(wins,loses, on="Agent", suffixes=["_wins","_loses"], how="outer")
winrate.fillna(0, inplace=True)
winrate["percent"] = winrate["count_wins"]/winrate["count_loses"]*50
winrate.replace(np.inf, 100, inplace=True)
winrate

,count_wins,count_loses,percent
Agent,,,
astra,207,218.0,47.477064
breach,42,47.0,44.680851
brimstone,11,5.0,110.000000
chamber,61,53.0,57.547170
cypher,81,81.0,50.000000
jett,278,272.0,51.102941
kayo,53,60.0,44.166667
killjoy,123,128.0,48.046875
omen,5,2.0,125.000000


### sending sql files

In [250]:
pickrate.to_sql(con=connect_sqlite, index=True, name="pickrate", if_exists="replace")
winrate.to_sql(con=connect_sqlite, index=True, name="winrate", if_exists="replace")

16